In [87]:
#  the libraries
import tensorflow as tf
import numpy as np
import librosa
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler

In [5]:
# drawing the distribution data

In [6]:
# drawing the spectogran contrast
#  refernce : https://huggingface.co/learn/audio-course/chapter1/audio_data
# drawing the waveform
# drawing the frequency spectrum

In [7]:
def check_total_wav(base_dir): 
    genres  = os.listdir(base_dir)
    iter = 0
    for genre in range(len(genres)): 
        for file in os.listdir((os.path.join(base_dir, genres[genre]))): 
            iter += 1
        print(f'total genre {genres[genre]} is :  {iter}')
        iter = 0
check_total_wav('./data/genres_original/')

total genre blues is :  99
total genre classical is :  99
total genre country is :  99
total genre disco is :  99
total genre hiphop is :  99
total genre jazz is :  99
total genre metal is :  99
total genre pop is :  99
total genre reggae is :  99
total genre rock is :  99


In [8]:
def random_check_result_dimension():
    random = "./data/genres_original/blues/blues.00000.wav"
    audio, sr  = librosa.load(random)
                    # chroma features
    array_cft = librosa.feature.chroma_stft(y=audio, sr=sr)
    array_zero_crossing_rate = librosa.feature.zero_crossing_rate(y  = audio )
    arrray_rms = librosa.feature.rms(y = audio)
    array_spectiaL_centroid = librosa.feature.spectral_centroid(y = audio)
    array_mfcc = librosa.feature.melspectrogram(y = audio, sr = sr)
    print(np.array(array_cft).shape)
    print(np.array(array_zero_crossing_rate).shape)
    print(np.array(arrray_rms).shape)
    print(np.array(array_spectiaL_centroid).shape)
    print(np.array(array_mfcc).shape)

random_check_result_dimension()

(12, 1293)
(1, 1293)
(1, 1293)
(1, 1293)
(128, 1293)


In [158]:
def get_average(array):
    array_flat = array.flatten()
    return np.mean(array_flat)

# list most important = pass
# chroma features = pass
# zero crossing rate  = pass
# rmse = pass
# spectiaL_centroid = pass
# MFCC  = pass
# the dataset is available on : https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification/data
# if neccessary
# cqt
# cens
# vqt
# melspectogram
# spectral_centroid
# spectral bandwidth 
# spectral_contrast
# the dataset is available on : https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification/data
def standart_scaler(data, columns):
    for column in columns: 
        mean = np.mean(np.array(data[column]))
        standart_deviasion = np.std(np.array(data[column]))
        if mean > 1:
            for row in range(data.shape[0]):
                #  just get two digits number after "."
                data.at[row, column] = np.round((data.loc[row, column]  -  mean) / standart_deviasion,  decimals = 3)
        else: 
            # print(f'masuk ke  : {column}')
            for row in range(data.shape[0]): 
                # print(f'ubah : {np.round(data.loc[row, column], decimals= 2)}')
                data.at[row, column] =  np.round(data.loc[row, column], decimals= 3)
    return data

#  function for convert to excel 
def convert_excel(data_extraction: dict): 
    df = pd.DataFrame(data_extraction)
    # df = standart_scaler(df, )
    df.to_excel("./data/extract_data/feature_extraction.xlsx")
#  succed



def high_level_feature_extraction(base_dir): 
    features = ['MFCC', 'MELSPECTOGRAM']
    classes = os.listdir("./data/genres_original/")

    # cretathe path
    for feature in features: 
        for class_i in classes: 
            if not os.path.exist(f'./data/{feature}/{class_i}'): 
                os.mkdir(f'./data/{feature}/{class_i}')

    for class_i  in  classes:
        full_path_load = f"./data/genres_original/{class_i}"
        for wav in full_path_load: 
            wav_load, sr = librosa.load(wav)
            mfcc = librosa.feature.mfcc(y=wav_load, sr=sr)
            mel_spectogram = librosa.feature.melspectrogram(y=wav_load, sr=sr)
            # save the result to image data
                



def low_feature_extract(base_dir): 
    # series_music = []
    labels = []
    iter_label = 0
    dict_features_extraction = {}
    # define array for feature extraction
    array_spectral_bandwidth = []
    array_zero_crossing_rate = []
    arrray_rms = []
    array_spectiaL_centroid = []
    array_mfcc = []
    array_wav  = []
    # the genres
    genres = os.listdir(base_dir)
    for genre in genres: 
        genre_path = os.path.join(base_dir, genre)
        # temp_series = []  
        for music in os.listdir(genre_path):
            try: 
                array_wav.append(music)
                audio, sr  = librosa.load(os.path.join(genre_path, music))
                # chroma features
                array_spectral_bandwidth.append(get_average(librosa.feature.spectral_bandwidth(y=audio, sr=sr)))
                # zero crossing rate
                array_zero_crossing_rate.append(get_average(librosa.feature.zero_crossing_rate(y  = audio)))
                # rms
                arrray_rms.append(get_average(librosa.feature.rms(y = audio))) 
                # spectral centroid
                array_spectiaL_centroid.append(get_average(librosa.feature.spectral_centroid(y = audio)))
                # mel spectorgram
                array_mfcc.append(get_average(librosa.feature.melspectrogram(y = audio, sr = sr)))
                # for labels
                labels.append(iter_label)
            except Exception as e: 
                # if audio data can't load
                print(f'{os.path.join(genre_path, music)}: {e}')
        # series_music.append(temp_series)
        iter_label += 1
    
    dict_features_extraction = {
        "wav" : array_wav, 
        "spactial bandwith" : array_spectral_bandwidth, 
        "zero crossing rate" : array_zero_crossing_rate,
        "rms" : arrray_rms, 
        "spactial centroid" : array_spectiaL_centroid,
        "mfcc" : array_mfcc, 
        "label" : labels
    }
    
    convert_excel(dict_features_extraction)
    # return the values
    print(f'finish convert to excel, please check to ./data/extract_data/feature_extraction.xlsx')

# load data that has been doing extraction 
def load_feature_extraction_data(data_path : str):
    # convert to excel 
    return pd.read_excel(data_path, index_col=0 ) 

# load data that has been doing extraction 
def get_train_test_data(df : pd.DataFrame): 
    features = df[df.columns[1:len(df.columns) - 1]]
    target = df[df.columns[-1]]
    x_train, y_train, x_test, y_test = train_test_split(features, target, random_state= 42, test_size=0.2)
    return x_train, y_train, x_test, y_test 

#  def norm_data()

def create_model_and_fit(x_train, y_train, x_test, y_test):
    # next : define callbacks

    class CustomCallbacks(tf.keras.callbacks.Callback): 
        def on_epochs_end(self, logs = {}): 
            if self.logs("accuracy") > 0.96: 
                self.stop_train = True

    callbacks = CustomCallbacks()
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units = 1024, activation = 'relu',  input_shape = (5,) ), 
        tf.keras.layers.Dense(units = 128, activation = 'relu'), 
        # tf.keras.layers.Dropout(rate = 0.2), 
        # tf.keras.layers.Dense(units = 16, activation = 'relu'), 
        tf.keras.layers.Dense(units = 10, activation = 'softmax'), 


    ])
    model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(), optimizer = "adam", metrics = ['accuracy'])
    history = model.fit(x_train, y_train, epochs = 500, validation_data = (x_test,  y_test),   callbacks = callbacks)
    return model, history 


In [38]:
low_feature_extract("./data/genres_original/")
data_train = load_feature_extraction_data("./data/extract_data/feature_extraction.xlsx")
data_train.head()

finish convert to excel, please check to ./data/extract_data/feature_extraction.xlsx


,wav,spactial bandwith,zero crossing rate,rms,spactial centroid,mfcc,label
0,blues.00000.wav,2002.412407,0.083045,0.130184,1784.122641,4.425542,0
1,blues.00001.wav,2038.987608,0.056040,0.095908,1530.261767,2.653440,0
2,blues.00002.wav,1747.754087,0.076291,0.175473,1552.832481,7.253840,0
3,blues.00003.wav,1596.422564,0.033309,0.141040,1070.153418,6.207545,0
4,blues.00004.wav,1748.410758,0.101461,0.091501,1835.128513,2.106331,0


In [159]:
df = load_feature_extraction_data("./data/extract_data/feature_extraction.xlsx")
df.head()

,wav,spactial bandwith,zero crossing rate,rms,spactial centroid,mfcc,label
0,blues.00000.wav,2002.412407,0.083045,0.130184,1784.122641,4.425542,0
1,blues.00001.wav,2038.987608,0.056040,0.095908,1530.261767,2.653440,0
2,blues.00002.wav,1747.754087,0.076291,0.175473,1552.832481,7.253840,0
3,blues.00003.wav,1596.422564,0.033309,0.141040,1070.153418,6.207545,0
4,blues.00004.wav,1748.410758,0.101461,0.091501,1835.128513,2.106331,0


In [160]:
# columns = df.columns
df = standart_scaler(df, [str(x) for x in df.columns[1 : -1]])
df.head()

,wav,spactial bandwith,zero crossing rate,rms,spactial centroid,mfcc,label
0,blues.00000.wav,-0.453,0.083,0.130,-0.580,-0.165,0
1,blues.00001.wav,-0.383,0.056,0.096,-0.935,-0.537,0
2,blues.00002.wav,-0.937,0.076,0.175,-0.903,0.430,0
3,blues.00003.wav,-1.225,0.033,0.141,-1.578,0.210,0
4,blues.00004.wav,-0.936,0.101,0.092,-0.509,-0.652,0


In [161]:
x_train, x_test,  y_train,y_test  = get_train_test_data(df)

In [162]:
model, history  = create_model_and_fit(x_train, y_train, x_test, y_test )

Epoch 1/500


c:\Users\Teddy Dharma\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.2604 - loss: 2.1359 - val_accuracy: 0.2273 - val_loss: 1.9989
Epoch 2/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3089 - loss: 1.8230 - val_accuracy: 0.3535 - val_loss: 1.8589
Epoch 3/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3456 - loss: 1.7327 - val_accuracy: 0.3333 - val_loss: 1.8487
Epoch 4/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3746 - loss: 1.6338 - val_accuracy: 0.3333 - val_loss: 1.9049
Epoch 5/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4280 - loss: 1.5752 - val_accuracy: 0.3434 - val_loss: 1.7605
Epoch 6/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4482 - loss: 1.5409 - val_accuracy: 0.3535 - val_loss: 1.7465
Epoch 7/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4209 - loss: 1.5153 - val_accuracy: 0.3586 - val_loss: 1.7274
Epoch 8/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4613 - loss: 1.4834 - val_accuracy: 0.3687 - val_loss: 1.7

In [191]:
# creating pretrained model using efficient net
random_var = random.randint(1, 700)
pred_data = df.iloc[random_var, 1:len(df.columns) - 1].to_numpy().astype(np.float32)

pred_data = np.expand_dims(pred_data, axis= 0)
print(np.argmax(model.predict(pred_data)))
print(f'actual : {df.iloc[random_var, -1]}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
8
actual : 5
